In [1]:
#import
import pandas as pd
import glob
import json
import os
import shutil
import time

from pyspark.sql import SparkSession
from pyspark.sql.functions import substring, length, col, expr, trim, sum
from pyspark.sql.types import IntegerType

#start timer
start_time = time.time()

# Create a Spark session
spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Artist Word Association") \
    .getOrCreate()

#read data
data1 = spark.read.json('data_split1.json', multiLine=True)
data2 = spark.read.json('data_split2.json', multiLine=True)
data3 = spark.read.json('data_split3.json', multiLine=True)
data4 = spark.read.json('data_split4.json', multiLine=True)

#select columns that we might need
deleteddf1 = data1.select('created_at', col("id").alias("tweet_id"), 'full_text', 'coordinates',
    'place','retweet_count', 'favorite_count','lang', 'entities.hashtags',
    'entities.symbols', 'entities.user_mentions', 'entities.urls', col('user.id').alias("userid"), 
    'user.name', 'user.screen_name', 'user.location', 'user.description', 'user.url', 
    'user.followers_count', 'user.profile_image_url_https', 'user.statuses_count')

deleteddf2 = data2.select('created_at', col("id").alias("tweet_id"), 'full_text', 'coordinates',
    'place','retweet_count', 'favorite_count','lang', 'entities.hashtags',
    'entities.symbols', 'entities.user_mentions', 'entities.urls', col('user.id').alias("userid"), 
    'user.name', 'user.screen_name', 'user.location', 'user.description', 'user.url', 
    'user.followers_count', 'user.profile_image_url_https', 'user.statuses_count')

deleteddf3 = data3.select('created_at', col("id").alias("tweet_id"), 'full_text', 'coordinates',
    'place','retweet_count', 'favorite_count','lang', 'entities.hashtags',
    'entities.symbols', 'entities.user_mentions', 'entities.urls', col('user.id').alias("userid"), 
    'user.name', 'user.screen_name', 'user.location', 'user.description', 'user.url', 
    'user.followers_count', 'user.profile_image_url_https', 'user.statuses_count')

deleteddf4 = data4.select('created_at', col("id").alias("tweet_id"), 'full_text', 'coordinates',
    'place','retweet_count', 'favorite_count','lang', 'entities.hashtags',
    'entities.symbols', 'entities.user_mentions', 'entities.urls', col('user.id').alias("userid"), 
    'user.name', 'user.screen_name', 'user.location', 'user.description', 'user.url', 
    'user.followers_count', 'user.profile_image_url_https', 'user.statuses_count')

#rename columns
deleteddf1 = deleteddf1.withColumn("created_at_month", substring("created_at", 5, 3)) \
    .withColumn("created_at_year", substring("created_at", 27, 4))
deleteddf1 = deleteddf1.drop('created_at')

deleteddf2 = deleteddf2.withColumn("created_at_month", substring("created_at", 5, 3)) \
    .withColumn("created_at_year", substring("created_at", 27, 4))
deleteddf2 = deleteddf2.drop('created_at')

deleteddf3 = deleteddf3.withColumn("created_at_month", substring("created_at", 5, 3)) \
    .withColumn("created_at_year", substring("created_at", 27, 4))
deleteddf3 = deleteddf3.drop('created_at')

deleteddf4 = deleteddf4.withColumn("created_at_month", substring("created_at", 5, 3)) \
    .withColumn("created_at_year", substring("created_at", 27, 4))
deleteddf4 = deleteddf4.drop('created_at')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/30 10:13:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


22/11/30 10:13:18 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [5]:
artistInfo_df1 = deleteddf1.select(col('name').alias('screenName'), col('screen_name').alias('handle'), col('profile_image_url_https').alias('URL')).distinct()
artistInfo_df2 = deleteddf2.select(col('name').alias('screenName'), col('screen_name').alias('handle'), col('profile_image_url_https').alias('URL')).distinct()
artistInfo_df3 = deleteddf3.select(col('name').alias('screenName'), col('screen_name').alias('handle'), col('profile_image_url_https').alias('URL')).distinct()
artistInfo_df4 = deleteddf4.select(col('name').alias('screenName'), col('screen_name').alias('handle'), col('profile_image_url_https').alias('URL')).distinct()

In [6]:
infomerge_df = artistInfo_df1.union(artistInfo_df2).union(artistInfo_df3).union(artistInfo_df4)

infomerge_df \
    .repartition(1) \
    .write.csv("infooutput", sep="\t", header=True, mode="overwrite")